## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-30-11-00-00 +0000,nypost,New $65K private school uses AI to teach stude...,https://nypost.com/2026/01/30/business/new-65k...
1,2026-01-30-10-52-57 +0000,bbc,Burkina Faso junta announces ban on all politi...,https://www.bbc.com/news/articles/cevnlv9my1no...
2,2026-01-30-10-46-47 +0000,nyt,Trump Says Putin Agreed to a Weeklong Pause in...,https://www.nytimes.com/2026/01/29/us/politics...
3,2026-01-30-10-42-00 +0000,wsj,Eurozone Economy Posts Resilient 2025 Growth D...,https://www.wsj.com/economy/frances-economy-sl...
4,2026-01-30-10-39-58 +0000,bbc,Syria's Kurds sense US betrayal as government ...,https://www.bbc.com/news/articles/cvgrzd76p4lo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2425/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,61
147,ice,24
0,new,17
365,he,14
140,minnesota,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2026-01-30-01-53-00 +0000,wsj,President Trump and Senate Democrats said ther...,https://www.wsj.com/politics/policy/white-hous...,146
230,2026-01-29-20-34-23 +0000,nypost,Trump prevents ‘boring’ cabinet meeting so he ...,https://nypost.com/2026/01/29/us-news/trump-sp...,120
83,2026-01-30-04-50-00 +0000,wsj,President Trump’s advisers have been told that...,https://www.wsj.com/economy/central-banking/tr...,118
88,2026-01-30-04-30-56 +0000,nyt,"Democrats Reach Spending Deal With Trump, Seek...",https://www.nytimes.com/2026/01/29/us/politics...,115
118,2026-01-30-02-02-45 +0000,startribune,The latest: Trump now says ‘nothing will chang...,https://www.startribune.com/ice-raids-minnesot...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,146,2026-01-30-01-53-00 +0000,wsj,President Trump and Senate Democrats said ther...,https://www.wsj.com/politics/policy/white-hous...
356,78,2026-01-29-13-31-00 +0000,wsj,Democratic Sen. Amy Klobuchar of Minnesota ann...,https://www.wsj.com/politics/elections/sen-amy...
0,69,2026-01-30-11-00-00 +0000,nypost,New $65K private school uses AI to teach stude...,https://nypost.com/2026/01/30/business/new-65k...
370,61,2026-01-29-12-00-01 +0000,startribune,AI image of Alex Pretti’s killing is the lates...,https://www.startribune.com/ai-image-of-alex-p...
160,46,2026-01-29-23-45-33 +0000,cbc,"Prime minister meets Heated Rivalry stars, pay...",https://www.cbc.ca/news/canada/livestory/heate...
367,46,2026-01-29-12-20-47 +0000,bbc,Nicki Minaj calls herself Trump's 'number one ...,https://www.bbc.com/news/articles/cp87l604nx8o...
79,44,2026-01-30-05-05-11 +0000,nypost,Trump threatens tariffs on any country selling...,https://nypost.com/2026/01/30/us-news/trump-th...
349,43,2026-01-29-14-00-00 +0000,latimes,"San José Mayor Matt Mahan, a Newsom critic and...",https://www.latimes.com/california/story/2026-...
272,43,2026-01-29-18-25-17 +0000,nypost,China executes 11 ‘key members’ linked to scam...,https://nypost.com/2026/01/29/world-news/11-sc...
264,39,2026-01-29-18-45-03 +0000,nypost,"Starbucks CEO lays out long-term growth plan, ...",https://nypost.com/2026/01/29/business/starbuc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
